# 🔍 Hybrid RAG System with Web Search

A **Hybrid Retrieval-Augmented Generation (RAG)** system that combines local document retrieval with real-time web search using Tavily API.

---

## 📋 Table of Contents

1. [Overview](#overview)
2. [Architecture](#architecture)
3. [Tech Stack](#tech-stack)
4. [Features](#features)
5. [How It Works](#how-it-works)
6. [Installation](#installation)

---

## 🎯 Overview

This notebook demonstrates a **Hybrid RAG** approach that:
- **Searches the web** for real-time information using Tavily
- **Retrieves from local documents** stored in ChromaDB
- **Generates answers** using Groq LLM (OpenAI GPT-OSS-120B)

---

## 🏗️ Architecture

### System Block Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         HYBRID RAG SYSTEM                                    │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                              INPUT LAYER                                     │
│                        ┌─────────────────────┐                              │
│                        │    User Question    │                              │
│                        │  "What is price of  │                              │
│                        │   iPhone 16?"       │                              │
│                        └──────────┬──────────┘                              │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                         HYBRID RETRIEVAL LAYER                               │
│                                                                              │
│   ┌─────────────────────────┐       ┌─────────────────────────┐            │
│   │   📚 LOCAL RETRIEVAL    │       │   🌐 WEB SEARCH         │            │
│   │                         │       │                         │            │
│   │  ┌─────────────────┐   │       │  ┌─────────────────┐   │            │
│   │  │   data.txt      │   │       │  │  Tavily API     │   │            │
│   │  │   (Source)      │   │       │  │  (Real-time)    │   │            │
│   │  └────────┬────────┘   │       │  └────────┬────────┘   │            │
│   │           ▼            │       │           ▼            │            │
│   │  ┌─────────────────┐   │       │  ┌─────────────────┐   │            │
│   │  │ Text Splitter   │   │       │  │ Search Query    │   │            │
│   │  │ (1024 chunks)   │   │       │  │                 │   │            │
│   │  └────────┬────────┘   │       │  └────────┬────────┘   │            │
│   │           ▼            │       │           ▼            │            │
│   │  ┌─────────────────┐   │       │  ┌─────────────────┐   │            │
│   │  │ Embeddings      │   │       │  │ Web Results     │   │            │
│   │  │ (MiniLM-L6-v2)  │   │       │  │ (JSON)          │   │            │
│   │  └────────┬────────┘   │       │  └────────┬────────┘   │            │
│   │           ▼            │       │           │            │            │
│   │  ┌─────────────────┐   │       │           │            │            │
│   │  │ ChromaDB        │   │       │           │            │            │
│   │  │ (Vector Store)  │   │       │           │            │            │
│   │  └────────┬────────┘   │       │           │            │            │
│   │           ▼            │       │           │            │            │
│   │  ┌─────────────────┐   │       │           │            │            │
│   │  │ Retriever (k=4) │   │       │           │            │            │
│   │  └────────┬────────┘   │       │           │            │            │
│   │           │            │       │           │            │            │
│   └───────────┼────────────┘       └───────────┼────────────┘            │
│               │                                │                          │
│               └────────────┬───────────────────┘                          │
│                            ▼                                              │
│                  ┌─────────────────┐                                      │
│                  │ Combined Context│                                      │
│                  └─────────────────┘                                      │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                           LLM GENERATION LAYER                               │
│                                                                              │
│   ┌─────────────────┐   ┌─────────────────┐   ┌─────────────────┐          │
│   │ Prompt Template │ → │   Groq LLM      │ → │ String Output   │          │
│   │                 │   │ (GPT-OSS-120B)  │   │ Parser          │          │
│   │ Context +       │   │                 │   │                 │          │
│   │ Question        │   │                 │   │                 │          │
│   └─────────────────┘   └─────────────────┘   └─────────────────┘          │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                              OUTPUT LAYER                                    │
│                        ┌─────────────────────┐                              │
│                        │   📝 AI Answer      │                              │
│                        │  "iPhone 16 price   │                              │
│                        │   in Nepal is..."   │                              │
│                        └─────────────────────┘                              │
└─────────────────────────────────────────────────────────────────────────────┘
```

### Data Flow Diagram

```
┌──────────────┐     ┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│   User       │     │   Hybrid     │     │   Prompt     │     │   Groq       │
│   Question   │ ──► │   Retrieval  │ ──► │   Template   │ ──► │   LLM        │
│              │     │              │     │              │     │              │
└──────────────┘     └──────────────┘     └──────────────┘     └──────────────┘
                            │                                         │
                            ▼                                         ▼
                     ┌──────────────┐                         ┌──────────────┐
                     │  Tavily API  │                         │   Answer     │
                     │  (Web Search)│                         │   Response   │
                     └──────────────┘                         └──────────────┘
```

### Component Interaction

```
┌────────────────────────────────────────────────────────────────────────────┐
│                              USER INTERFACE                                 │
│                         (Jupyter Notebook Cell)                             │
└────────────────────────────────────────────────────────────────────────────┘
                                     │
                                     ▼
┌────────────────────────────────────────────────────────────────────────────┐
│                              rag_answer()                                   │
│  ┌──────────────────────────────────────────────────────────────────────┐  │
│  │  1. Call hybrid_retrieval(question)                                   │  │
│  │  2. Build prompt with context                                         │  │
│  │  3. Send to LLM via chain                                             │  │
│  │  4. Parse and return response                                         │  │
│  └──────────────────────────────────────────────────────────────────────┘  │
└────────────────────────────────────────────────────────────────────────────┘
                                     │
                    ┌────────────────┴────────────────┐
                    ▼                                 ▼
┌───────────────────────────────┐   ┌───────────────────────────────────────┐
│      hybrid_retrieval()       │   │           LangChain Chain              │
│  ┌─────────────────────────┐  │   │  ┌─────────────────────────────────┐  │
│  │ web_search_tool.run()   │  │   │  │  prompt | llm | StrOutputParser │  │
│  │         │               │  │   │  └─────────────────────────────────┘  │
│  │         ▼               │  │   │                  │                    │
│  │  ┌─────────────────┐    │  │   │                  ▼                    │
│  │  │  Tavily API     │    │  │   │  ┌─────────────────────────────────┐  │
│  │  │  (Web Search)   │    │  │   │  │        ChatGroq LLM             │  │
│  │  └─────────────────┘    │  │   │  │   (openai/gpt-oss-120b)         │  │
│  └─────────────────────────┘  │   │  └─────────────────────────────────┘  │
└───────────────────────────────┘   └───────────────────────────────────────┘
```

---

## 🛠️ Tech Stack

| Component | Technology | Purpose |
|-----------|------------|---------|
| **Web Search** | Tavily API | Real-time internet search |
| **Text Splitting** | RecursiveCharacterTextSplitter | Chunk documents |
| **Embeddings** | sentence-transformers/all-MiniLM-L6-v2 | Text to vectors |
| **Vector Store** | ChromaDB | Store and search embeddings |
| **LLM** | Groq (openai/gpt-oss-120b) | Generate responses |
| **Framework** | LangChain | RAG pipeline orchestration |

---

## ✨ Features

| Feature | Description |
|---------|-------------|
| 🌐 **Web Search** | Real-time search using Tavily API |
| 📚 **Local Retrieval** | Search local documents in ChromaDB |
| 🔀 **Hybrid Approach** | Combines web + local context |
| 🤖 **LLM Generation** | Answers using GPT-OSS-120B via Groq |
| ⚡ **Fast Inference** | Groq's LPU for quick responses |

---

## 🔧 How It Works

### Step 1: Web Search Tool Setup
```python
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import Tool

tavily = TavilySearchResults()
web_search_tool = Tool(
    name="web_search",
    func=tavily.run,
    description="Search the internet for real-time information."
)
```

### Step 2: Local Document Processing
```python
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

# Embeddings
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Split docs
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
docs = splitter.split_documents([Document(page_content=open("data.txt").read())])

# Save to Chroma
vector_db = Chroma.from_documents(docs, emb)
retriever = vector_db.as_retriever(search_kwargs={"k": 4})
```

### Step 3: LLM Setup
```python
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)
```

### Step 4: Hybrid Retrieval
```python
def hybrid_retrieval(query):
    web_result = web_search_tool.run(query)
    context = str(web_result)
    return context
```

### Step 5: RAG Answer Generation
```python
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
You are a helpful AI assistant.

Use the context below:

{context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

def rag_answer(question):
    ctx = hybrid_retrieval(question)
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"context": ctx, "question": question})
```

---

## 📦 Installation

### Requirements

```bash
pip install langchain langchain-community langchain-groq langchain-huggingface
pip install langchain-chroma chromadb sentence-transformers
pip install tavily-python
```

### Environment Variables

```bash
# .env file
GROQ_API_KEY=your_groq_api_key
TAVILY_API_KEY=your_tavily_api_key
```

---

## 📊 Configuration

| Parameter | Value | Description |
|-----------|-------|-------------|
| Chunk Size | 1024 | Characters per chunk |
| Chunk Overlap | 200 | Overlapping characters |
| Retriever k | 4 | Top documents to retrieve |
| LLM Temperature | 0 | Deterministic output |
| LLM Model | openai/gpt-oss-120b | 120B parameter model |
| Embedding Model | all-MiniLM-L6-v2 | 384 dimensions |

---

## 🚀 Usage Example

```python
# Ask a question
question = "What is the price of iPhone 16 in Nepal?"
answer = rag_answer(question)
print(answer)
```

---

## 📈 Model Information

### Groq LLM: openai/gpt-oss-120b

| Property | Value |
|----------|-------|
| **Model Name** | openai/gpt-oss-120b |
| **Parameters** | 120 Billion |
| **Provider** | Groq |
| **Temperature** | 0 (Deterministic) |
| **Use Case** | Question Answering, RAG |

---

**Made with ❤️ using LangChain, Tavily & Groq**

In [4]:
# from langchain_community.tools.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import Tool

tavily = TavilySearchResults()

web_search_tool = Tool(
    name="web_search",
    func=tavily.run,
    description="Search the internet for real-time information."
)


/var/folders/wk/485j8zms4wx1g2b_h5h_gxn80000gn/T/ipykernel_16326/1815874624.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document


# Embeddings
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Split docs
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
docs = splitter.split_documents([Document(page_content=open("data.txt").read())])

# Save to Chroma
vector_db = Chroma.from_documents(docs, emb)


In [22]:
retriever = vector_db.as_retriever(search_kwargs={"k": 4})


In [23]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)


In [ ]:
def hybrid_retrieval(query):
        web_result = web_search_tool.run(query)
        context = str(web_result)

        return context


In [25]:
hybrid_retrieval(query="What is the price of iPhone 16 in Nepal?")

'[{\'title\': \'iPhone 16 Price in Nepal, Launch, Specifications, Availability\', \'url\': \'https://www.gadgetbytenepal.com/iphone-16-price-nepal/\', \'content\': "The latest official price of the iPhone 16 in Nepal is NPR 135,299 for the 128GB variant, while its 256GB and 512GB models aren\'t available anymore. And if you", \'score\': 0.9604838}, {\'title\': \'Apple iPhone Price in Nepal 2025 [Updated]\', \'url\': \'https://www.gadgetbytenepal.com/apple-iphone-price-nepal/\', \'content\': \'Price in Nepal: NPR 135,299 (128GB). Read full specs of iPhone 16 here. 6. iPhone 16e. iPhone\', \'score\': 0.92731744}, {\'title\': \'Apple iPhones Price in Nepal 2025 | Buy Now | Hukut.com\', \'url\': \'https://hukut.com/mobile-phones/apple\', \'content\': \'Apple iPhone 16 Pro\\n\\nRs. 1,88,200\\n\\nRs. 1,97,999\\n\\nSave Rs. 9,799\\n\\n0.0\\n\\nOut Of Stock\\n\\n+1 Offers for you >\\n\\nApple iPhone 16 Pro Max\\n\\nRs. 2,07,000\\n\\n5.0\\n\\nOut Of Stock\\n\\nHome [...] Save Rs. 21,890\\n\\n0.

In [26]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
You are a helpful AI assistant.

Use the context below:

{context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

def rag_answer(question):
    ctx = hybrid_retrieval(question)
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"context": ctx, "question": question})


In [27]:
print(rag_answer("What is the price of iPhone 16 in Nepal?"))


The current official price of the iPhone 16 in Nepal is **NPR 135,299** for the 128 GB model. (The 256 GB and 512 GB variants are no longer listed.)【source】(https://www.gadgetbytenepal.com/iphone-16-price-nepal/)
